In [1]:
# Read WAV files to array
from pydub import AudioSegment
import numpy as np
from scipy.io import wavfile
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import plotly
import os
import soundfile as sf

# read WAV file using scipy.io.wavfile
fs_wav, data_wav = wavfile.read(os.path.join('E:/Files','Acoustic Data', '0.3Eru_0.85Gul_1.15Gra_2.45Rum_4.1Moo.wav')) #fs_wav means 采样率（sampling rate）; data_wav means 音频样本数据

with sf.SoundFile(os.path.join('E:/Files','Acoustic Data', '0.3Eru_0.85Gul_1.15Gra_2.45Rum_4.1Moo.wav'), 'r') as audio_file:
    channels = audio_file.channels
    bit_depth = audio_file.subtype
    print(f"channel: {channels}")
    print(f"bit_depth: {bit_depth}")

print('Signal Duration = {} seconds'.format(data_wav.shape[0] / fs_wav))

e:\Python\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


channel: 1
bit_depth: PCM_16
Signal Duration = 4.103375 seconds


C:\Users\34691\AppData\Local\Temp\ipykernel_19076\379976145.py:15: WavFileWarning:

Chunk (non-data) not understood, skipping it.



In [ ]:
# Normalization
data_wav_norm = data_wav / (2**16) #2**n depends on bit_depth; audio file is normalized into [-1,1]
time_wav = np.arange(0, len(data_wav)) / fs_wav #Create a timeline that matches the length of the audio data
plotly.offline.iplot({ "data": [go.Scatter(x=time_wav, 
                                           y=data_wav_norm, 
                                           name='normalized audio signal')]})

In [4]:
# Trim (segment) audio signal (10 seconds)
data_wav_norm_crop = data_wav_norm[0 * fs_wav: 10 * fs_wav]
time_wav_crop = np.arange(0, len(data_wav)) / fs_wav
plotly.offline.iplot({ "data": [go.Scatter(x=time_wav_crop, 
                                           y=data_wav_norm_crop, 
                                           name='cropped audio signal')]})

In [ ]:
###切割音频
### Fix-sized segmentation (breaks a signal into non-overlapping segments)
fs_wav, data_wav = wavfile.read(os.path.join('E:/Files','Acoustic Data','Parts','part_audio_000.wav'))
signal = data_wav / (2**8)
signal_len = len(signal) #Get the length of the audio signal, i.e. the number of samples
segment_size_t = 1 # segment size in seconds
segment_size = segment_size_t * fs_wav  # segment size in samples
### Break signal into list of segments in a single-line Python code
segments = np.array([signal[x:x + segment_size] for x in
                     np.arange(0, signal_len, segment_size)])
### Save each segment in a seperate filename
for iS, s in enumerate(segments):
    wavfile.write("data/obama_segment_{0:d}_{1:d}.wav".format(segment_size_t * iS,
                                                              segment_size_t * (iS + 1)), fs_wav, (s))

In [ ]:
###从录音中删除无声片段的简单算法
import IPython
# Remove pauses using an energy threshold = 50% of the median energy:
energies = [(s**2).sum() / len(s) for s in segments]
# (attention: integer overflow would occure without normalization here!)
thres = 0.5 * np.median(energies)# Half of the median of all segmented energies
index_of_segments_to_keep = (np.where(energies > thres)[0])
# get segments that have energies higher than a the threshold:
segments2 = segments[index_of_segments_to_keep]
# concatenate segments to signal:
new_signal = np.concatenate(segments2)
# and write to file:
wavfile.write("data/obama_processed.wav", fs_wav, new_signal)
plotly.offline.iplot({ "data": [go.Scatter(y=energies, name="energy"),
                                go.Scatter(y=np.ones(len(energies)) * thres, 
                                           name="thres")]})
# play the initial and the generated files in notebook:
IPython.display.display(IPython.display.Audio("data/obama.wav"))
IPython.display.display(IPython.display.Audio("data/obama_processed.wav"))